In [ ]:
import rasterio
import os
from datetime import datetime
import numpy as np

In [ ]:
# local libraries
from coupling_PCR_FM.model_functions_v2 import PCR_model, CMF_model, DFM_model
from coupling_PCR_FM.utils import config_to_dict
%config Application.log_level="INFO"

In [ ]:
# IMPORT MODEL SETTINGS FROM INI-FILE
argv1 = r'default_rename.set'
argv2 = r'paths.env'

# parse set/ini-file with central/general settings for coupling framework
config = config_to_dict(argv1)
# parse env-file for user-specific paths and environmental variables
envs = config_to_dict(argv2)
# combine
config.update(envs)
options = config
# parse dates
start_date = datetime.strptime(options['numerical_settings']['startTime'], '%Y-%m-%d')
end_date = datetime.strptime(options['numerical_settings']['endTime'], '%Y-%m-%d')

In [ ]:
# setup output dir
cwd = os.getcwd() # note: this get changed by pcr initialization later on
out_dir = options['PCRpaths']['outputDirectoryPCR']
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
print(out_dir)
print(cwd)

## create PCR model BMI object

In [ ]:
PCR_config_fn = os.path.join(cwd, options['hydrologic_model']['config_dir'], options['hydrologic_model']['config_file'])
PCR_in_dir = options['PCRpaths']['inputDirectoryPCR']
PCR_out_dir = os.path.join(out_dir, 'PCR')
print(PCR_in_dir)
print(PCR_config_fn)
print(PCR_out_dir)

PCR_bmi = PCR_model(PCR_config_fn, PCR_in_dir, PCR_out_dir,
                            start_date, end_date,)


## create DFM model BMI object

In [ ]:
DFM_engine = os.path.join(cwd, options['DFM_engine']['DFM_path'])
DFM_model_dir = os.path.join(cwd, options['hydrodynamic_model']['model_dir'])
DFM_config_fn = os.path.join(DFM_model_dir, options['hydrodynamic_model']['model_file'])
DFM_out_dir = os.path.join(out_dir, 'DFM')
print(DFM_engine)
print(DFM_model_dir)
print(DFM_config_fn)
print(DFM_out_dir)


DFM_bmi = DFM_model(DFM_engine, DFM_config_fn, DFM_model_dir, DFM_out_dir,
                              start_date, end_date)

## intitialize DFM model first to expose 1d coords

In [ ]:
# initialize DFM to expose model coordinates via BMI
DFM_bmi.initialize()

## set coupling between PCR and DFM

Couple external 1d coordinates to internal model 2d grid. 

The model routing is deactivated at coupled cells and area fractions are calculated to distribute water volume accross 1d nodes.

In [ ]:
PCR_bmi.couple_grid_to_1d(DFM_bmi)

In [ ]:
# initialize PCR to after editing ldd grid
PCR_bmi.initialize()

## run coupled PCR - DFM model

In [ ]:
# run coupled model for 2 timesteps
DFMidx = DFM_bmi.coupled_idx
PCRidx = PCR_bmi.coupled_idx
area = DFM_bmi.get_var('ba')[DFMidx]

# discharge at non-active ldd cells == runoff
def update_states():
    delta_water_volume_pcr = PCR_bmi.get_var('discharge')[PCRidx] * PCR_bmi.options['dt'] # [m3/dt]
    delta_water_depth_dfm = delta_water_volume_pcr * area_frac / area # [m/dt]
    DFM_bmi.set_var_index('rain', DFMidx, delta_water_depth_dfm)
        
for i in range(2):
    PCR_bmi.update()
    update_states()
    DFM_bmi.update()   

In [ ]:
DFM_bmi.finalize()
PCR_bmi.finalize()